In [299]:
import csv
import json
import math
import os
import random
from datetime import timedelta

import numpy as np


class Customer(object):
    def __init__(self, customer_id, loyalty_score):
        self.customer_id = customer_id
        self.value_score = loyalty_score


def generate_customers(output_location_root, number_of_customers, return_data=True):
    customers = []
    with open(f'{output_location_root}/customers.csv', mode='w') as customers_file:
        csv_writer = csv.writer(customers_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(["customer_id", "loyalty_score"])
        for cid in range(1, number_of_customers + 1):
            score = np.random.randint(low=1, high=11)
            customer_id = f"C{cid}"
            csv_writer.writerow([customer_id, score])
            if return_data:
                customers.append(Customer(customer_id, score))
    return customers if return_data else None


def generate_products(output_location_root, products_to_generate):
    product_count_digits = int(math.log10(len(sum(products_to_generate.values(), []))) + 1)

    product_id_lookup = {k: {} for k, v in products_to_generate.items()}
    with open(f'{output_location_root}/products.csv', mode='w') as products_file:
        csv_writer = csv.writer(products_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(["product_id", "product_description", "product_category"])
        item_index = 1
        for category in products_to_generate:
            for item in products_to_generate[category]:
                product_id = f"P{str(item_index).zfill(product_count_digits)}"
                csv_writer.writerow([product_id, item, category])
                product_id_lookup[category][item] = product_id
                item_index += 1
    return product_id_lookup


def generate_transactions(output_location_root, customers, products, product_id_lookup, products_cats_frequency,
                          start_datetime, end_datetime):
    open_files = open_transaction_sinks(output_location_root, start_datetime, end_datetime)
    product_cats_count = len(products.keys())
    num_days = (end_datetime - start_datetime).days
    all_days = [start_datetime + timedelta(days=d) for d in range(0, num_days + 1)]
    customer_frequency_type = [int(num_days / 14), int(num_days / 10), int(num_days / 7), int(num_days / 5),
                               int(num_days / 4), int(num_days / 3)]

    for customer in customers:
        num_transaction_days = random.choice(customer_frequency_type)
        num_cats = random.randint(1, product_cats_count)
        customer_transaction_days = sorted(random.sample(all_days, num_transaction_days))
        cats = random.sample(products_cats_frequency, num_cats)
        for day in customer_transaction_days:
            transaction = {
                "customer_id": customer.customer_id,
                "basket": generate_basket(products, product_id_lookup, cats),
                "date_of_purchase": str(day + timedelta(minutes=random.randint(168, 1439)))
            }
            open_files[to_canonical_date_str(day)].write(json.dumps(transaction) + "\n")

    for f in open_files.values():
        f.close()


def to_canonical_date_str(date_to_transform):
    return date_to_transform.strftime('%Y-%m-%d')


def open_transaction_sinks(output_location_root, start_datetime, end_datetime):
    root_transactions_dir = f"{output_location_root}/transactions/"
    open_files = {}
    days_to_generate = (end_datetime - start_datetime).days
    for next_day_offset in range(0, days_to_generate + 1):
        next_day = to_canonical_date_str(start_datetime + timedelta(days=next_day_offset))
        day_directory = f"{root_transactions_dir}/d={next_day}"
        os.makedirs(day_directory, exist_ok=True)
        open_files[next_day] = open(f"{day_directory}/transactions.json", mode='w')
    return open_files


def generate_basket(products, product_id_lookup, cats):
    num_items_in_basket = random.randint(1, 3)
    basket = []
    product_category = random.choice(cats)
    for item in [random.choice(products[product_category]) for _ in range(0, num_items_in_basket)]:
        product_id = product_id_lookup[product_category][item]
        basket.append({
            "product_id": product_id,
            "price": random.randint(1, 2000)
        })
    return basket


In [300]:
import os
import numpy as np

from datetime import datetime

from data_generator import generate_customers, generate_products, generate_transactions

if __name__ == "__main__":
    np.random.seed(seed=42)

    products_data = {
        "house": ["detergent", "kitchen roll", "bin liners", "shower gel", "scented candles", "fabric softener",
                  "cling film", "aluminium foil", "toilet paper", "kitchen knife", "dishwasher tablets", "ice pack"],
        "clothes": ["men's dark green trousers", "women's shoes", "jumper", "men's belt", "women's black socks",
                    "men's striped socks", "men's trainers", "women's blouse", "women's red dress"],
        "fruit_veg": ["avocado", "cherries", "scotch bonnets", "peppers", "broccoli", "potatoes", "grapes",
                      "easy peeler", "mango", "lemon grass", "onions", "apples", "raspberries"],
        "sweets": ["carrot cake", "salted caramel dark chocolate", "gummy bears", "kombucha", "ice cream", "irn bru"],
        "food": ["steak", "chicken", "mince beef", "milk", "hummus", "activated charcoal croissant", "whole chicken",
                 "tuna", "smoked salmon", "camembert", "pizza", "oats", "peanut butter", "almond milk", "lentil soup",
                 "greek yoghurt", "parmesan", "coconut water", "chicken stock",  "water"],
        "bws": ["red wine", "gin", "cognac", "cigarettes"]
    }
    products_cats_frequency = ["house"]*15 + ["clothes"]*5 + ["fruit_veg"]*25 + ["sweets"] * 20 + ["food"] * 25 + \
                              ["bws"] * 10

    gen_id = "starter"
    output_location = f"../input_data/{gen_id}"
    os.makedirs(output_location, exist_ok=True)

    gen_customers = generate_customers(output_location, 137)
    product_id_lookup = generate_products(output_location, products_data)

    start_date = datetime(2018, 12, 1, 0, 0, 0)
    end_date = datetime(2019, 3, 1, 23, 59, 59)
    generate_transactions(output_location, gen_customers, products_data, product_id_lookup, products_cats_frequency,
                          start_date, end_date)


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [1]:
cus_data = pd.read_csv("customers.csv")

<IPython.core.display.Javascript object>

In [3]:
prod_data = pd.read_csv("products.csv")

<IPython.core.display.Javascript object>

In [4]:
prod_data

product_id product_description product_category
0         P01           detergent            house
1         P02        kitchen roll            house
2         P03          bin liners            house
3         P04          shower gel            house
4         P05     scented candles            house
..        ...                 ...              ...
59        P60               water             food
60        P61            red wine              bws
61        P62                 gin              bws
62        P63              cognac              bws
63        P64          cigarettes              bws

[64 rows x 3 columns]

In [5]:
cus_data

customer_id  loyalty_score
0            C1              7
1            C2              4
2            C3              8
3            C4              5
4            C5              7
..          ...            ...
132        C133              3
133        C134              6
134        C135              2
135        C136             10
136        C137              9

[137 rows x 2 columns]

In [ ]:
df_list = []
for i in [x[0] for x in os.walk('starter/transactions/')]:
    if i != 'starter/transactions/':
        df_list.append(pd.read_json(i+'/transactions.json',lines = True))
Trans_data = pd.concat(df_list)

In [25]:
Trans_data

customer_id                                             basket  \
0           C7             [{'product_id': 'P26', 'price': 1277}]   
1          C11  [{'product_id': 'P38', 'price': 285}, {'produc...   
2          C20  [{'product_id': 'P48', 'price': 117}, {'produc...   
3          C26  [{'product_id': 'P11', 'price': 1248}, {'produ...   
4          C27             [{'product_id': 'P25', 'price': 1258}]   
..         ...                                                ...   
27        C114              [{'product_id': 'P49', 'price': 637}]   
28        C116              [{'product_id': 'P12', 'price': 209}]   
29        C127  [{'product_id': 'P54', 'price': 111}, {'produc...   
30        C132  [{'product_id': 'P01', 'price': 99}, {'product...   
31        C137  [{'product_id': 'P01', 'price': 858}, {'produc...   

       date_of_purchase  
0   2018-12-01 14:07:00  
1   2018-12-01 09:01:00  
2   2018-12-01 10:01:00  
3   2018-12-01 14:10:00  
4   2018-12-01 05:21:00  
..                  ...  
27  2019-03-01 05:51:00  
28  2019-03-01 11:39:00  
29  2019-03-01 05:37:00  
30  2019-03-01 18:45:00  
31  2019-03-01 05:21:00  

[2190 rows x 3 columns]

In [26]:
Trans_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2190 entries, 0 to 31
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customer_id       2190 non-null   object
 1   basket            2190 non-null   object
 2   date_of_purchase  2190 non-null   object
dtypes: object(3)
memory usage: 68.4+ KB


In [29]:
Trans_cus_data =pd.merge(Trans_data, cus_data, on = 'customer_id', how = 'left')

In [30]:
Trans_cus_data

customer_id                                             basket  \
0             C7             [{'product_id': 'P26', 'price': 1277}]   
1            C11  [{'product_id': 'P38', 'price': 285}, {'produc...   
2            C20  [{'product_id': 'P48', 'price': 117}, {'produc...   
3            C26  [{'product_id': 'P11', 'price': 1248}, {'produ...   
4            C27             [{'product_id': 'P25', 'price': 1258}]   
...          ...                                                ...   
2185        C114              [{'product_id': 'P49', 'price': 637}]   
2186        C116              [{'product_id': 'P12', 'price': 209}]   
2187        C127  [{'product_id': 'P54', 'price': 111}, {'produc...   
2188        C132  [{'product_id': 'P01', 'price': 99}, {'product...   
2189        C137  [{'product_id': 'P01', 'price': 858}, {'produc...   

         date_of_purchase  loyalty_score  
0     2018-12-01 14:07:00              3  
1     2018-12-01 09:01:00              4  
2     2018-12-01 10:01:00              2  
3     2018-12-01 14:10:00              1  
4     2018-12-01 05:21:00             10  
...                   ...            ...  
2185  2019-03-01 05:51:00              3  
2186  2019-03-01 11:39:00              5  
2187  2019-03-01 05:37:00              4  
2188  2019-03-01 18:45:00              7  
2189  2019-03-01 05:21:00              9  

[2190 rows x 4 columns]

In [51]:
Trans_cus_data['basket'].iloc[100][0]['product_id']

'P11'

In [62]:
df = pd.DataFrame()
cus_id = []
prod_id = []
price = []
date_perch = []
loyalty = []
for i in Trans_cus_data.index:
    if len(Trans_cus_data['basket'][i]) ==1:
        cus_id.append(Trans_cus_data['customer_id'][i])
        prod_id.append(Trans_cus_data['basket'][i][0]['product_id'])
        price.append(Trans_cus_data['basket'][i][0]['price'])
        date_perch.append(Trans_cus_data['date_of_purchase'][i])
        loyalty.append(Trans_cus_data['loyalty_score'][i])
    elif len(Trans_cus_data['basket'][i]) ==2:
        cus_id.append(Trans_cus_data['customer_id'][i])
        prod_id.append(Trans_cus_data['basket'][i][0]['product_id'])
        price.append(Trans_cus_data['basket'][i][0]['price'])
        date_perch.append(Trans_cus_data['date_of_purchase'][i])
        loyalty.append(Trans_cus_data['loyalty_score'][i])
        
        cus_id.append(Trans_cus_data['customer_id'][i])
        prod_id.append(Trans_cus_data['basket'][i][1]['product_id'])
        price.append(Trans_cus_data['basket'][i][1]['price'])
        date_perch.append(Trans_cus_data['date_of_purchase'][i])
        loyalty.append(Trans_cus_data['loyalty_score'][i])
    elif len(Trans_cus_data['basket'][i]) ==3:
        cus_id.append(Trans_cus_data['customer_id'][i])
        prod_id.append(Trans_cus_data['basket'][i][0]['product_id'])
        price.append(Trans_cus_data['basket'][i][0]['price'])
        date_perch.append(Trans_cus_data['date_of_purchase'][i])
        loyalty.append(Trans_cus_data['loyalty_score'][i])
        
        cus_id.append(Trans_cus_data['customer_id'][i])
        prod_id.append(Trans_cus_data['basket'][i][1]['product_id'])
        price.append(Trans_cus_data['basket'][i][1]['price'])
        date_perch.append(Trans_cus_data['date_of_purchase'][i])
        loyalty.append(Trans_cus_data['loyalty_score'][i])
        
        cus_id.append(Trans_cus_data['customer_id'][i])
        prod_id.append(Trans_cus_data['basket'][i][2]['product_id'])
        price.append(Trans_cus_data['basket'][i][2]['price'])
        date_perch.append(Trans_cus_data['date_of_purchase'][i])
        loyalty.append(Trans_cus_data['loyalty_score'][i])
    
df['customer_id'] = cus_id
df['product_id'] = prod_id
df['price'] = price
df['date_of_purchase'] = date_perch
df['loyalty_score'] = loyalty

In [65]:
Trans_cus_prod_data = pd.merge(df, prod_data, on = 'product_id', how = 'left')

In [67]:
Trans_cus_prod_data.to_csv('final_df.csv')